<a href="https://colab.research.google.com/github/RyanTokManMokMTM/NLP_Final/blob/main/nlp_getting_started.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Kaggle Compitition with NLP
## Natural Language Processing with Disaster Tweets

### loading the data and check datas format

In [6]:
#load data csv with pd
import pandas as pd
train_data = pd.read_csv("train.csv",encoding="utf-8")
test_data = pd.read_csv("test.csv",encoding="utf-8")
submission_data = pd.read_csv("sample_submission.csv",encoding="utf-8")

#### see what we got

In [7]:
#train data
train_data[:10]

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
6,10,NaN,NaN,#flood #disaster Heavy rain causes flash flood...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
8,14,NaN,NaN,There's an emergency evacuation happening now ...,1
9,15,NaN,NaN,I'm afraid that the tornado is coming to our a...,1


In [8]:
#shape of the data
train_data.shape

(7613, 5)

In [9]:
#header of the data
print(train_data.columns.get_value)

<bound method Index.get_value of Index(['id', 'keyword', 'location', 'text', 'target'], dtype='object')>


In [27]:
#Drop the column that we don't use it
train_data.drop(['id', 'keyword', 'location'],axis=1)

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1
...,...,...
7608,Two giant cranes holding a bridge collapse int...,1
7609,@aria_ahrary @TheTawniest The out of control w...,1
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...,1
7611,Police investigating after an e-bike collided ...,1


#### TODO


*   Clean Data(remove all html stopword etc)   





In [10]:
#check the target
count = train_data.target.value_counts() #return how many unique target
display(count)

0    4342
1    3271
Name: target, dtype: int64

In [11]:
train_data.loc[0].text

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [35]:
#split feature and label
features = train_data["text"].tolist()
label = train_data["target"].tolist()

#### Define some function to remove the html url etc

In [12]:
import re

In [13]:
#remove the url link in the text
def urlRemover(txt):
  #read the url with regex 
  urlStr = re.compile("https?://\S+|www\.\S+")
  return urlStr.sub("",txt);

funcTester = "i like google : https://www.google.com.hk"
print(urlRemover(funcTester))

i like google : 


In [14]:
#remove the html tag
def htmlTagRemover(html):
  htmlTxt = re.compile("<.*?>")
  return htmlTxt.sub("",html)

test = """<div>
<h1>Real or Fake</h1>
<p>Kaggle </p>
<a href="https://www.kaggle.com/c/nlp-getting-started">getting started</a>
</div>"""
print(htmlTagRemover(test))


Real or Fake
Kaggle 
getting started



In [15]:
#remove all the emoji in txt
def emojiRemover(emojiTxt):
  '''
    Here will define some unicode bounds
    emoticons,symbols,transport,and other
  '''
  emoji= re.compile("["
      u"\U0001F600-\U0001F64F"
      u"\U0001F300-\U0001F5FF"
      u"\U0001F680-\U0001F6FF"
      u"\U0001F1E0-\U0001F1FF"
      u"\U00002702-\U000027B0"
      u"\U000024C2-\U0001F251"
      "]+",flags = re.UNICODE
    )
  return emoji.sub("",emojiTxt)

print(emojiRemover("tesing emoji 😞 😟 😠"))

tesing emoji   


In [83]:
#define a clean text function
def clean_texts(text):
  '''
  textList: 
   a list for all text has to clean up
  1.change all text to lower
  2.remove url
  3.remove html
  4.remove emoji
  5.return the text
  '''
  text = text.lower()
  #clean all not a-zA-z0-9 text
  textRe = re.compile("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?")
  return textRe.sub("",text)

In [84]:
#testing
clean_feature_data = []
for txt in features:
  clean_feature_data.append(clean_texts(txt))

In [85]:
clean_feature_data[:100]

['our deeds are the reason of this earthquake may allah forgive us all',
 'forest fire near la ronge sask canada',
 'all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected',
 '13000 people receive wildfires evacuation orders in california ',
 'just got sent this photo from ruby alaska as smoke from wildfires pours into a school ',
 'rockyfire update  california hwy 20 closed in both directions due to lake county fire  cafire wildfires',
 'flood disaster heavy rain causes flash flooding of streets in manitou colorado springs areas',
 'im on top of the hill and i can see a fire in the woods',
 'theres an emergency evacuation happening now in the building across the street',
 'im afraid that the tornado is coming to our area',
 'three people died from the heat wave so far',
 'haha south tampa is getting flooded hah wait a second i live in south tampa what am i gonna do what am i gonna do fvck flooding',
 'raining 

#### need to clean all stop word with cleaning word list
#### stopword list with NL Toolkit stopword.word(english)

In [86]:
#download the stopword list from nltk package
import nltk.corpus
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [87]:
#load import nltk stopwords english word
from nltk.corpus import stopwords
stopwordsList = stopwords.words("english")

In [88]:
#all english stop list
stopwordsList

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [89]:
#clean all stop words in each sentance
def clean_stop_word(cleanTxt):
  return " ".join([word for word in cleanTxt.split() if not word in stopwordsList])

In [90]:
all_clean_text = []
for sentance in clean_feature_data:
  all_clean_text.append(clean_stop_word(sentance))

In [91]:
all_clean_text[:10]

['deeds reason earthquake may allah forgive us',
 'forest fire near la ronge sask canada',
 'residents asked shelter place notified officers evacuation shelter place orders expected',
 '13000 people receive wildfires evacuation orders california',
 'got sent photo ruby alaska smoke wildfires pours school',
 'rockyfire update california hwy 20 closed directions due lake county fire cafire wildfires',
 'flood disaster heavy rain causes flash flooding streets manitou colorado springs areas',
 'im top hill see fire woods',
 'theres emergency evacuation happening building across street',
 'im afraid tornado coming area']

In [136]:
#splite our data to train data and test data
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(all_clean_text,label,random_state = 0)

#### Basic SGDClassifer

In [122]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfTransformer
sgd = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf',  TfidfTransformer()),
    ('nb', SGDClassifier()),
])
#fit the data to model
model = sgd.fit(x_train, y_train)

In [129]:
from sklearn.metrics import classification_report
#try to predict the model
predict = model.predict(x_test)

#print the predict ans
print(classification_report(y_test,predict))

              precision    recall  f1-score   support

           0       0.80      0.85      0.83      1107
           1       0.78      0.71      0.74       797

    accuracy                           0.79      1904
   macro avg       0.79      0.78      0.78      1904
weighted avg       0.79      0.79      0.79      1904



## Tokenization
#### make the word as a vector of each sentence

In [140]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

#initial the CountVectorizer
vectorizer = CountVectorizer(analyzer="word",tokenizer=None,preprocessor=None,stop_words=None)

#fit_transform will fit the data to model and learn the word
train_feature = vectorizer.fit_transform(x_train)
test_feature = vectorizer.transform(x_test)
train_feature = train_feature.toarray()
test_feature = test_feature.toarray()

In [104]:
#feature shape 
train_feature.shape #(total sentence,total vector(verb))

(7613, 15828)

In [130]:
#all word will be tokenized
vectorizer.get_feature_names()

['0011',
 '001116',
 '0025',
 '005225',
 '010156',
 '010217',
 '0104',
 '010401',
 '0106',
 '0111',
 '012032',
 '012624',
 '015025',
 '02',
 '0206',
 '0215',
 '03',
 '0306',
 '0308',
 '030811',
 '034',
 '0400',
 '045',
 '05',
 '05082015',
 '05th',
 '06',
 '0605',
 '061',
 '063243',
 '063424',
 '06jst',
 '07',
 '070',
 '0700',
 '0730',
 '075',
 '080',
 '0800',
 '080215',
 '0802pm',
 '080515',
 '080615',
 '08062015',
 '08072015',
 '08315',
 '0840728dynamix',
 '09',
 '0913',
 '0936',
 '0day',
 '0npzp',
 '10',
 '100',
 '1000',
 '10000',
 '100000',
 '1000amp1000',
 '1000s',
 '1008pla',
 '1008planet',
 '100bn',
 '100mb',
 '100nd',
 '100s',
 '101',
 '1015',
 '1015pm',
 '1028',
 '103',
 '1030',
 '1030pm',
 '103424',
 '1038pm',
 '1040',
 '10401',
 '1040pm',
 '105',
 '106',
 '1061',
 '10638',
 '107',
 '1079',
 '109',
 '10am',
 '10k',
 '10km',
 '10m',
 '10monthold',
 '10news',
 '10pm',
 '10th',
 '10thcr',
 '10w',
 '10x',
 '11',
 '1100',
 '11000',
 '11000000',
 '110358',
 '111020',
 '1115',
 '112'

#### Tring on RandomForestClassifier

In [143]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)

#fit moodel with tokenization 
rfc = rfc.fit(train_feature,y_train)

In [145]:
#now train to predict
rdf_Predict = rfc.predict(test_feature)

In [147]:
print(classification_report(y_test,rdf_Predict))

              precision    recall  f1-score   support

           0       0.78      0.92      0.85      1107
           1       0.85      0.65      0.74       797

    accuracy                           0.81      1904
   macro avg       0.82      0.78      0.79      1904
weighted avg       0.81      0.81      0.80      1904



#### Tring on Bert Model

In [148]:
!pip install transformers 

     |████████████████████████████████| 2.6 MB 5.2 MB/s 
     |████████████████████████████████| 3.3 MB 49.8 MB/s 
     |████████████████████████████████| 895 kB 73.5 MB/s 
     |████████████████████████████████| 636 kB 75.4 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [157]:
#import bertModel
from transformers import BertTokenizer,BertConfig,TFBertModel

#load bert tokenizer and model
tokenizer = BertTokenizer.from_pretrained("https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt")
config = BertConfig.from_pretrained("https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json")
bert_model = TFBertModel.from_pretrained("https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-tf_model.h5",config=config)
train_encode = tokenizer.encode(x_train,truncation=True,padding=True,max_length=128)
test_encode = tokenizer.encode(x_test,truncation=True,padding=True,max_length=128)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1641: FutureWarning: Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated and won't be possible anymore in v5. Use a model identifier or the path to a directory instead.
  FutureWarning,


Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

Some layers from the model checkpoint at https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-tf_model.h5 were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-tf_model.h5.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [159]:
#vector < 128 add 0 as padding
#train data
import numpy as np
train_ids = []
train_mask = []
train_token = []
for text in all_clean_text:
  token = tokenizer.encode(text,max_length=128)
  padding_size = 128 - len(token)
  train_ids.append(token + [0]*padding_size)
  train_mask.append([1]*len(token) + [0]*padding_size) #only token is 1 other padding to 0
  train_token.append([0]*128) #all 0
train_ids = np.array(train_ids)
train_mask = np.array(train_mask)
train_token = np.array(train_token)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [168]:
#change label to array
train_label = np.array(label)

In [169]:
train_label

array([1, 1, 1, ..., 1, 1, 1])

In [167]:
print(train_ids[8])
print(train_mask[8])
print(train_token[8])

[  101  2045  2015  5057 13982  6230  2311  2408  2395   102     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0]
[1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
#now create a model and train
import keras
input = keras.layers.Input(shape=(128,),dtype="int32")
mask = keras.layers.Input(shape=(128),dtype="int32")
token = keras.layers.Input(shape=(128),dtype="int32")

#bert model
_ , x = bert_model([input,mask,token])#input 3 arrgument id mast and token_ids
outputs = keras.layers.Dense(1,"sigmoid")(x)
model = keras.models.Model(input=[input,mask,token],outputs=outputs)

#compile the model
model.compile(loss="binary_crossentropy",optimizer="Adam",metrics=['accuracy'])

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
